In [24]:
import json
from PIL import Image
import csv
import glob
import os

In [25]:
# 저장할 디렉토리 설정
save_directory = './sample_data/preprocessed_images'

# 저장 디렉토리가 없으면 생성
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [26]:
# 이미지 파일이 위치한 디렉토리
image_directory = './sample_data/image/'

In [27]:
# CSV 파일을 위한 헤더
csv_headers = ['filename', 'label']

# 저장된 이미지의 개수를 추적하는 변수
saved_images_count = 0

# 이미지 파일이 위치한 디렉토리
image_directory = './sample_data/images/'

# 저장할 디렉토리 설정
save_directory = 'cropped_images'

# 저장 디렉토리가 없으면 생성
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# JSON 파일들을 먼저 읽어서 전체 처리해야 할 항목 수를 파악합니다.
total_items_to_process = 0
for json_file_path in glob.glob('./sample_data/label/*.json'):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
    total_items_to_process += len(json_data)

# CSV 파일 생성
with open('labels.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_headers)
    
    # json_files 폴더 내의 모든 .json 파일에 대해 반복
    for json_file_path in glob.glob('./sample_data/label/*.json'):
        # JSON 파일을 읽고 Python dictionary로 변환
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        for item in json_data:
            # 원본 이미지 파일 이름
            original_filename = item['filename']
            # '\\' 앞부분까지의 이름 추출
            base_pattern = original_filename.split('\\')[0]
            
            # 해당 패턴을 포함하는 모든 파일 찾기 (하위 디렉토리 포함)
            matching_files = glob.glob(f"{image_directory}**/{base_pattern}*", recursive=True)
            
            if matching_files:
                # 일치하는 파일이 있다면, 첫 번째 파일 사용
                image_path = matching_files[0]
            else:
                # 일치하는 파일이 없다면, 다음 항목으로 넘어감
                print(f"No matching file found for pattern {base_pattern}")
                continue
            
            # 얼굴 표정 라벨
            label = item['annot_A']['faceExp']
            
            # 이미지 불러오기
            try:
                with Image.open(image_path) as img:
                    # Bounding box 좌표 추출
                    boxes = item['annot_A']['boxes']
                    minX, minY, maxX, maxY = boxes['minX'], boxes['minY'], boxes['maxX'], boxes['maxY']
                    
                    # 이미지 Crop
                    cropped_img = img.crop((minX, minY, maxX, maxY))
                    
                    # Crop된 이미지 저장 경로 설정
                    cropped_filepath = os.path.join(save_directory, os.path.basename(image_path))
                    
                    # Crop된 이미지 저장
                    cropped_img.save(cropped_filepath)
                    
                    # CSV 파일에 파일명과 라벨 쓰기
                    writer.writerow([cropped_filepath, label])
                    
                    # 이미지가 성공적으로 저장되면 카운터 증가
                    saved_images_count += 1
                    
                    # 진행률 출력
                    progress = (saved_images_count / total_items_to_process) * 100
                    print(f"Progress: {progress:.2f}%, Saved images: {saved_images_count}/{total_items_to_process}")
                    
            except IOError:
                print(f"Error opening or processing image {image_path}")

# 저장된 이미지의 총 개수 출력
print(f"Total saved images: {saved_images_count}")

Total saved images: 2290


In [20]:
import cv2
import tempfile
import shutil
import numpy as np

In [28]:
def list_images_without_faces(directory):
    no_faces_files = []  # 얼굴이 감지되지 않은 이미지의 파일명을 저장할 리스트

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(directory, filename)
            
            try:
                img_pil = Image.open(path)
            except IOError:
                print(f"Error loading image {filename}. It may be corrupted or format may not be supported.")
                continue

            img = np.array(img_pil)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            
            if len(faces) == 0:
                no_faces_files.append(filename)  # 얼굴이 감지되지 않은 이미지의 파일명을 리스트에 추가

    # 얼굴이 감지되지 않은 이미지의 파일명과 총 개수 출력
    for file_name in no_faces_files:
        print(f'{file_name} has no detected faces.')
    print(f'Total images without detected faces: {len(no_faces_files)}')

# 저장된 이미지가 있는 디렉토리 경로 지정
saved_images_directory = './sample_data/preprocessed_images'

# 얼굴이 감지되지 않은 이미지 리스트 출력 실행
list_images_without_faces(saved_images_directory)

064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_공공시설&종교&의료시설_20201206203554-001-008.jpg has been deleted because no faces were detected.
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-034.jpg has been deleted because no faces were detected.
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-039.jpg has been deleted because no faces were detected.
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-044.jpg has been deleted because no faces were detected.
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_숙박 및 거주공간_20201206203716-010-004.jpg has been deleted because no faces were detected.
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_숙박 및 거주공간_20201206203716-010-006.jpg has been deleted because no faces were detected.
064c9650db76c75f0d8cdc31898eb031aece3cd1